## stuff langchian을 직접 구현
LCEL (LangChian Expression Langauge)를 이용해서 직접 stuff chain을 구현해보기

Lang Chain의 구성요소
- prompt
- retriever
-- question을 받고 vector store에서  document list를 반환 
- llm, chatmodel
- tool
- outputparser

In [7]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import Chroma ## vector store
from langchain.vectorstores import FAISS ## vector store
from langchain.schema import Document
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

cache_dir = LocalFileStore("../../db/vector-store/")

llm = ChatOpenAI(
    temperature=0.1
)
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./file/little_prince.txt")

docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cached_embeddins = CacheBackedEmbeddings.from_bytes_store(embeddings,cache_dir)
vectorstore = FAISS.from_documents(docs,cached_embeddins)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",),
        ("human", "{question}"),
    ]
)

chain = {"context" :retriever, "question" : RunnablePassthrough()} | prompt | llm

result = chain.invoke("Describe fox")

print(result.content)

The fox in the story is initially described as a creature that desires to be tamed by the little prince. It explains the concept of taming as establishing ties and forming a unique bond with someone. The fox expresses that through taming, both parties become special to each other in the whole world. It also mentions that taming involves patience and observing proper rites. The fox values the time spent with the little prince and appreciates the unique connection they have formed.
